In [1]:
import pandas as pd
news_file = r'D:\Data\NR_data\dev\news.tsv'

news_df = pd.read_table(news_file,index_col=None,names=['newsID','category','subcategory','title','abstract','url','entity_title','entity_abstract'])

In [8]:
def news_token_generator(news_file):
    news_df = pd.read_table(news_file,index_col=None,names=['newsID','category','subcategory','title','abstract','url','entity_title','entity_abstract'])
    news_iterator = news_df.iterrows()

    tokenizer = get_tokenizer('basic_english')

    for _,i in news_iterator:
        yield tokenizer(i['title'])

In [6]:
news_token_generator(news_file)

<generator object news_token_generator at 0x000001F59FE561C8>

In [10]:
from torchtext.vocab import build_vocab_from_iterator
a = build_vocab_from_iterator(news_token_generator(news_file))

51282lines [00:12, 4048.96lines/s]


1